### Домашнее задание

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)

5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
6. применить random negative sampling для построения классификатора в новых условиях
7. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
8. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

Источник датасета https://www.kaggle.com/ronitf/heart-disease-uci


Описание:

1. age - age in years
2. sex - (1 = male; 0 = female)
3. chest pain type (4 values)
4. resting blood pressure - in mm Hg on admission to the hospital
5. serum cholestoral in mg/dl 
6. fasting blood sugar > 120 mg/dl - 1 = true; 0 = false
7. resting electrocardiographic results - values 0,1,2
8. maximum heart rate achieved 
9. exercise induced angina - (1 = yes; 0 = no) 
10. oldpeak = ST depression induced by exercise relative to rest
11. the slope of the peak exercise ST segment 
12. ca - number of major vessels (0-3) colored by flourosopy
13. thal - 3 = normal; 6 = fixed defect; 7 = reversable defect


In [268]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

data = pd.read_csv("heart.csv")
data.head(3)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1


In [269]:
data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [270]:
len(data.loc[data['age'] <=19]) #юношеского возраста нет

0

In [271]:
cat_1 = len(data.loc[data['age'] <= 35]) #молодые
cat_2 = len(data.loc[(data['age'] > 35) & (data['age'] <= 50)])  #зрелые первая половина
cat_3 = len(data.loc[(data['age'] > 50) & (data['age'] < 60)]) #зрелые вторая половина
cat_4 = len(data.loc[(data['age'] >= 60) & (data['age'] < 76)]) # пожилые
cat_5 = len(data.loc[data['age'] >= 76]) # очень пожилые
cat_1 + cat_2 + cat_3 + cat_4 + cat_5 

303

In [272]:
data.loc[data['age'] <= 35, 'age'] = 1
data.loc[(data['age'] > 35) & (data['age'] <= 48), 'age'] = 2
data.loc[(data['age'] > 48) & (data['age'] < 60), 'age'] = 3
data.loc[(data['age'] >= 60) & (data['age'] < 76), 'age'] = 4 
data.loc[data['age'] >= 76, 'age'] = 5
data['age'].value_counts()

3    130
4     88
2     76
1      7
5      2
Name: age, dtype: int64

In [273]:
cat_1 = len(data.loc[data['trestbps'] < 100])  #гипотония
cat_2 = len(data.loc[(data['trestbps'] >= 100) & (data['trestbps'] < 130)]) #норма
cat_3 = len(data.loc[(data['trestbps'] >= 130) & (data['trestbps'] < 140)]) #повышенное
cat_4 = len(data.loc[(data['trestbps'] >= 140) & (data['trestbps'] < 160)])  #гипертония 1 степени
cat_5 = len(data.loc[(data['trestbps'] >= 160) & (data['trestbps'] < 180)])  #гипертония 2 степени
cat_6 = len(data.loc[data['trestbps'] >= 180])  #гипертония 3 степени

cat_1 + cat_2 + cat_3 + cat_4 + cat_5 + cat_6

303

In [274]:
data.loc[data['trestbps'] < 100, 'trestbps'] = 1 
data.loc[(data['trestbps'] >= 100) & (data['trestbps'] < 130), 'trestbps'] = 2
data.loc[(data['trestbps'] >= 130) & (data['trestbps'] < 140), 'trestbps'] = 3
data.loc[(data['trestbps'] >= 140) & (data['trestbps'] < 160), 'trestbps'] = 4
data.loc[(data['trestbps'] >= 160) & (data['trestbps'] < 180), 'trestbps'] = 5
data.loc[data['trestbps'] >= 180, 'trestbps'] = 6
data['trestbps'].value_counts()

2    133
4     71
3     71
5     21
6      5
1      2
Name: trestbps, dtype: int64

In [275]:
data.loc[data['chol'] < 200, 'chol'] = 1 # желательный 
data.loc[(data['chol'] >= 200) & (data['chol'] < 240), 'chol'] = 2 #  пограничный
data.loc[data['chol'] >= 240, 'chol'] = 3 #  высокий
data['chol'].value_counts()

3    155
2     98
1     50
Name: chol, dtype: int64

In [276]:
data.loc[data['thalach'] < 70, 'thalach'] = 1 # низкий
data.loc[(data['thalach'] >= 70) & (data['thalach'] < 100), 'thalach'] = 2 #  номальный
data.loc[data['thalach'] >= 100, 'thalach'] = 3 #  высокий
data['thalach'].value_counts()

3    295
2      8
Name: thalach, dtype: int64

In [277]:
data['target'].value_counts()

1    165
0    138
Name: target, dtype: int64

In [278]:
data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,3.006601,0.683168,0.966997,2.970297,2.346535,0.148515,0.528053,2.973597,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,0.813762,0.466011,1.032052,1.062193,0.747211,0.356198,0.525860,0.160595,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,3.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,3.000000,1.000000,1.000000,3.000000,3.000000,0.000000,1.000000,3.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,4.000000,1.000000,2.000000,4.000000,3.000000,0.000000,1.000000,3.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,5.000000,1.000000,3.000000,6.000000,3.000000,1.000000,2.000000,3.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


Разбиваем выборку на тренировочную и тестовую части и обучаем модель (в примере - градиентный бустинг)

In [279]:
from sklearn.model_selection import train_test_split

x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [280]:
model = GradientBoostingClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

Проверяем качество

In [281]:
results = {
    'approach': [],
    'f1': [],
    'roc': [],
    'recall':[],
    'precision':[]
}

from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 



results['approach'].append('gbc')
results['f1'].append(f1_score(y_test, y_predict))
results['roc'].append(roc_auc_score(y_test, y_predict))
results['recall'].append(recall_score(y_test, y_predict, average='binary'))
results['precision'].append(precision_score(y_test, y_predict, average='binary'))

evaluate_results(y_test, y_predict)

Classification results:
f1: 78.79%
roc: 76.94%
recall: 83.87%
precision: 74.29%


### Теперь очередь за PU learning

Представим, что нам неизвестны негативы и часть позитивов

In [282]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 42/165 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [283]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    261
 1     42
Name: class_test, dtype: int64


In [284]:
mod_data.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,class_test
0,4,1,3,4,2,1,0,3,0,2.3,0,0,1,1,-1
1,2,1,2,3,3,0,1,3,0,3.5,0,0,2,1,-1
2,2,0,1,3,2,0,0,3,0,1.4,2,0,2,1,1
3,3,1,1,2,2,0,1,3,0,0.8,2,0,2,1,1
4,3,0,0,2,3,0,1,3,1,0.6,2,0,2,1,-1
5,3,1,0,4,1,0,1,3,0,0.4,1,0,1,1,-1
6,3,0,1,4,3,0,0,3,0,1.3,1,0,2,1,1
7,2,1,1,2,3,0,1,3,0,0.0,2,0,3,1,-1
8,3,1,2,5,1,1,1,3,0,0.5,2,0,3,1,1
9,3,1,2,4,1,0,1,3,0,1.6,2,0,2,1,-1


Remember that this data frame (x_data) includes the former target variable that we keep here just to compare the results

[:-2] is the original class label for positive and negative data [:-1] is the new class for positive and unlabeled data

In [285]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

### 1. random negative sampling 25%

In [286]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(42, 15) (42, 15)


In [287]:
model = GradientBoostingClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

results['approach'].append('gbc_25%_rns')
results['f1'].append(f1_score(sample_test.iloc[:,-2].values, y_predict))
results['roc'].append(roc_auc_score(sample_test.iloc[:,-2].values, y_predict))
results['recall'].append(recall_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))
results['precision'].append(precision_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))


evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 79.15%
roc: 78.34%
recall: 90.29%
precision: 70.45%


### 2. random negative sampling 10%

In [288]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 10% of the positives marked
pos_sample_len = int(np.ceil(0.1 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 17/165 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [289]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    286
 1     17
Name: class_test, dtype: int64


Remember that this data frame (x_data) includes the former target variable that we keep here just to compare the results

[:-2] is the original class label for positive and negative data [:-1] is the new class for positive and unlabeled data

In [290]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [291]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(17, 15) (17, 15)


In [292]:
model = GradientBoostingClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

results['approach'].append('gbc_10%_rns')
results['f1'].append(f1_score(sample_test.iloc[:,-2].values, y_predict))
results['roc'].append(roc_auc_score(sample_test.iloc[:,-2].values, y_predict))
results['recall'].append(recall_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))
results['precision'].append(precision_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))


evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 70.59%
roc: 66.16%
recall: 77.70%
precision: 64.67%


### 3. random negative sampling 50%

In [293]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 10% of the positives marked
pos_sample_len = int(np.ceil(0.5 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 83/165 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [294]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    220
 1     83
Name: class_test, dtype: int64


In [295]:
mod_data.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,class_test
0,4,1,3,4,2,1,0,3,0,2.3,0,0,1,1,-1
1,2,1,2,3,3,0,1,3,0,3.5,0,0,2,1,-1
2,2,0,1,3,2,0,0,3,0,1.4,2,0,2,1,-1
3,3,1,1,2,2,0,1,3,0,0.8,2,0,2,1,-1
4,3,0,0,2,3,0,1,3,1,0.6,2,0,2,1,-1
5,3,1,0,4,1,0,1,3,0,0.4,1,0,1,1,1
6,3,0,1,4,3,0,0,3,0,1.3,1,0,2,1,1
7,2,1,1,2,3,0,1,3,0,0.0,2,0,3,1,1
8,3,1,2,5,1,1,1,3,0,0.5,2,0,3,1,1
9,3,1,2,4,1,0,1,3,0,1.6,2,0,2,1,-1


Remember that this data frame (x_data) includes the former target variable that we keep here just to compare the results

[:-2] is the original class label for positive and negative data [:-1] is the new class for positive and unlabeled data

In [296]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [297]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(83, 15) (83, 15)


In [298]:
model = GradientBoostingClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

results['approach'].append('gbc_50%_rns')
results['f1'].append(f1_score(sample_test.iloc[:,-2].values, y_predict))
results['roc'].append(roc_auc_score(sample_test.iloc[:,-2].values, y_predict))
results['recall'].append(recall_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))
results['precision'].append(precision_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))


evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 70.49%
roc: 76.83%
recall: 87.76%
precision: 58.90%


In [299]:
pd.DataFrame(data=results)

,approach,f1,roc,recall,precision
0,gbc,0.787879,0.769355,0.838710,0.742857
1,gbc_25%_rns,0.791489,0.783353,0.902913,0.704545
2,gbc_10%_rns,0.705882,0.661566,0.776978,0.646707
3,gbc_50%_rns,0.704918,0.768321,0.877551,0.589041
